In [2]:
import os
from pathlib import Path
from typing import List

from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


In [11]:
def parse_medical_txt(file_path: Path, year: int) -> List[Document]:
    """
    Parse a single year's medical dialogue txt file.
    Each dialogue record becomes ONE Document.
    """
    text = file_path.read_text(encoding="utf-8")

    # 以 id= 作为强分隔
    raw_records = text.split("\nid=")

    documents = []

    for i, record in enumerate(raw_records):
        record = record.strip()
        if not record:
            continue

        # 把被 split 掉的 id= 补回来
        if not record.startswith("id="):
            record = "id=" + record

        documents.append(
            Document(
                page_content=record,
                metadata={
                    "year": year,
                    "source_file": file_path.name,
                    "record_index": i
                }
            )
        )

    return documents


In [10]:
from pathlib import Path
from typing import List
from langchain.schema import Document

def load_all_medical_dialogues(base_dir: Path, year: int = None) -> List[Document]:
    """
    Load medical dialogues from the base directory.

    Args:
        base_dir: Path to the folder containing yearly txt files (2010.txt ... 2020.txt)
        year: Optional, only load this specific year if provided

    Returns:
        List of Documents
    """
    all_docs = []

    years_to_load = [year] if year is not None else range(2010, 2021)

    for y in years_to_load:
        file_path = base_dir / f"{y}.txt"
        if not file_path.exists():
            print(f"[WARN] Missing file: {file_path}")
            continue

        year_docs = parse_medical_txt(file_path, y)
        all_docs.extend(year_docs)

        print(f"[INFO] Loaded {len(year_docs)} records from {y}.txt")

    print(f"[INFO] Total records: {len(all_docs)}")
    return all_docs


In [6]:
os.environ["OPENAI_API_KEY"] = "sk-O7h0gucD1ZXKzC9A11z8aMBGuqAU7472gqhfHfP8Ff0Q51Qd"
os.environ["OPENAI_BASE_URL"] = "https://api.chatanywhere.tech"

In [12]:
from pathlib import Path
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from tqdm import tqdm
import time

# ==========================
# Paths
# ==========================

PROJECT_ROOT = Path.cwd().parent   # 当前 notebook / 脚本所在目录的上一级
DATA_DIR = PROJECT_ROOT / "original_data" / "Medical_Dialogue"

VECTOR_DIR = PROJECT_ROOT / "vector_store_Medical_Dialogue" / "faiss"
VECTOR_DIR.mkdir(parents=True, exist_ok=True)

print("Data directory:", DATA_DIR)
print("Vector store directory:", VECTOR_DIR)

# ==========================
# Load documents
# ==========================

print("\n Step 1: Loading medical dialogue documents...")
start_time = time.time()

documents = load_all_medical_dialogues(DATA_DIR, year=2011)

print(f" Loaded {len(documents)} dialogue records")
print(f" Time cost: {time.time() - start_time:.2f} seconds")

# 可选：快速 sanity check
print("\n Example document preview:")
print(documents[0].page_content[:500])
print("Metadata:", documents[0].metadata)

# ==========================
# Embedding model
# ==========================

print("\n Step 2: Initializing embedding model...")

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

print(" Embedding model initialized")

# ==========================
# Build FAISS with progress
# ==========================

print("\n Step 3: Building FAISS vector store (with progress)...")

start_time = time.time()

# 手动分批，便于debug

batch_size = 64
vectorstore = None

for i in tqdm(range(0, len(documents), batch_size), desc="🔄 Embedding batches"):
    batch_docs = documents[i:i + batch_size]

    if vectorstore is None:
        vectorstore = FAISS.from_documents(batch_docs, embeddings)
    else:
        vectorstore.add_documents(batch_docs)

print(f"\n FAISS index built with {len(documents)} documents")
print(f"  Total embedding time: {time.time() - start_time:.2f} seconds")

# ==========================
# Save FAISS locally
# ==========================

print("\n Step 4: Saving FAISS vector store to disk...")

vectorstore.save_local(str(VECTOR_DIR))

print(" FAISS vector store saved successfully!")
print(" Location:", VECTOR_DIR.resolve())


Data directory: e:\python\PyCharm Community Edition 2023.2.1\Medical_Dialogue_Summary\original_data\Medical_Dialogue
Vector store directory: e:\python\PyCharm Community Edition 2023.2.1\Medical_Dialogue_Summary\vector_store_Medical_Dialogue\faiss

 Step 1: Loading medical dialogue documents...
[INFO] Loaded 79712 records from 2011.txt
[INFO] Total records: 79712
 Loaded 79712 dialogue records
 Time cost: 34.79 seconds

 Example document preview:
id=1
https://www.haodf.com/wenda/drxiaohui_g_637628385.htm

Doctor faculty
首都医科大学附属北京同仁医院 普外科

Description
疾病：
乳腺纤维腺瘤 
病情描述（发病时间、主要症状、就诊医院等）：
从发现第一个小硬块起至今至少一年了吧。彩超检查出至少三个硬块，其中一个硬块较大，按压时会有轻微痛感。不按不疼。
曾经治疗情况和效果：
今年第一次做乳腺检查。
想得到怎样的帮助：
想咨询下医生，我这种情况需要如何治疗，是否需要动手术？如需要手术，是否适合到2012年5月份左右再做？还是做这个手术是否需要避开经期？什么时间最合适？

Dialogue
医生：
建议手术切除，可以5月份再手术。
Metadata: {'year': 2011, 'source_file': '2011.txt', 'record_index': 1}

 Step 2: Initializing embedding model...
 Embedding model initialized

 Step 3: Building FAISS vector store (with progress)...


🔄 Embedding batches:   1%|          | 10/1246 [03:00<6:10:51, 18.00s/it]


KeyboardInterrupt: 

In [13]:
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from tqdm import tqdm
import time

# ==========================
# Paths
# ==========================

PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "original_data" / "Medical_Dialogue"
VECTOR_DIR = PROJECT_ROOT / "vector_store_Medical_Dialogue" / "faiss_chunks_2011"
VECTOR_DIR.mkdir(parents=True, exist_ok=True)

file_2011 = DATA_DIR / "2011.txt"
if not file_2011.exists():
    raise FileNotFoundError(f"{file_2011} not found")

print("Data file:", file_2011)
print("Vector store directory:", VECTOR_DIR)

# ==========================
# Load full text
# ==========================

print("\n Step 1: Loading 2011 text as a single string...")
with open(file_2011, "r", encoding="utf-8") as f:
    full_text = f.read()

print(f" Total characters in file: {len(full_text)}")

# ==========================
# Split into chunks
# ==========================

print("\n Step 2: Splitting text into chunks...")
# 可调参数：chunk_size, chunk_overlap
chunk_size = 1000     # 每个 chunk 字符数
chunk_overlap = 200   # chunk 重叠部分字符数

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

texts = text_splitter.split_text(full_text)

# 转成 Document 对象
documents = [Document(page_content=t, metadata={"source_file": "2011.txt", "chunk_index": i})
             for i, t in enumerate(texts)]

print(f" Total chunks created: {len(documents)}")
print(" Example chunk preview:\n", documents[0].page_content[:500])

# ==========================
# Embedding model
# ==========================

print("\n Step 3: Initializing embedding model...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print(" Embedding model initialized")

# ==========================
# Build FAISS vector store with progress
# ==========================

batch_size = 64
vectorstore = None
start_time = time.time()

for i in tqdm(range(0, len(documents), batch_size), desc="🔄 Embedding batches"):
    batch_docs = documents[i:i + batch_size]

    if vectorstore is None:
        vectorstore = FAISS.from_documents(batch_docs, embeddings)
    else:
        vectorstore.add_documents(batch_docs)

print(f"\nFAISS built with {len(documents)} chunks")
print(f"Total embedding time: {time.time() - start_time:.2f} seconds")

# ==========================
# Save locally
# ==========================

vectorstore.save_local(str(VECTOR_DIR))
print("FAISS vector store saved at:", VECTOR_DIR.resolve())


Data file: e:\python\PyCharm Community Edition 2023.2.1\Medical_Dialogue_Summary\original_data\Medical_Dialogue\2011.txt
Vector store directory: e:\python\PyCharm Community Edition 2023.2.1\Medical_Dialogue_Summary\vector_store_Medical_Dialogue\faiss_chunks_2011

 Step 1: Loading 2011 text as a single string...
 Total characters in file: 34075516

 Step 2: Splitting text into chunks...
 Total chunks created: 44495
 Example chunk preview:
 id=1
https://www.haodf.com/wenda/drxiaohui_g_637628385.htm

Doctor faculty
首都医科大学附属北京同仁医院 普外科

Description
疾病：
乳腺纤维腺瘤 
病情描述（发病时间、主要症状、就诊医院等）：
从发现第一个小硬块起至今至少一年了吧。彩超检查出至少三个硬块，其中一个硬块较大，按压时会有轻微痛感。不按不疼。
曾经治疗情况和效果：
今年第一次做乳腺检查。
想得到怎样的帮助：
想咨询下医生，我这种情况需要如何治疗，是否需要动手术？如需要手术，是否适合到2012年5月份左右再做？还是做这个手术是否需要避开经期？什么时间最合适？

Dialogue
医生：
建议手术切除，可以5月份再手术。

id=2
https://www.haodf.com/wenda/lijianguo615_g_622324433.htm

Doctor faculty
青岛大学附属医院 乳腺病诊疗中心

Description
疾病：
乳腺纤维腺瘤 
病情描述（发病时间、主要症状、就诊医院等）：
从发现第一个小硬块起至今至少一年了吧

 Step 3: Initializing embedding model...
 Embedding mod

🔄 Embedding batches:   0%|          | 1/696 [00:26<5:12:39, 26.99s/it]


KeyboardInterrupt: 